In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import urllib
from tqdm.notebook import tqdm
from pathlib import Path
from multiprocessing import Pool
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchtext

import timm
import fastai.text.all as fastai
from fastai.callback.schedule import Learner

# from torch.utils.tensorboard import SummaryWriter
from tensorboardX import SummaryWriter
%load_ext tensorboard

import spacy
import re
import nltk

seed = 42
np.random.seed(seed + 1)
torch.manual_seed(seed)

In [ ]:
def get_device():
    if torch.cuda.is_available():
        device = 'cuda'
    elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
        device = 'mps'
    else:
        device = 'cpu'
    return device

device = get_device()

print(f'device: {device}')

In [ ]:
def get_optimizer(model, optimizer='sgd', lr=0.001, weight_decay=0, momentum=0, betas=(0.9, 0.999), eps=1e-8):
    if optimizer == 'sgd':
        opt = torch.optim.SGD(
            model.parameters(),
            lr=lr, 
            weight_decay=weight_decay, 
            momentum=momentum
        )
    if optimizer == 'adam':
        opt = torch.optim.Adam(
            model.parameters(),
            lr=lr,
            weight_decay=weight_decay,
            betas=betas,
            eps=eps
        )
    return opt

def train_classifier(train_data, model, opt, loss_fn, test_data=None, num_epochs=10, plot_loss=True, batch_size=32,
                     tensorboard=False, print_stats=True, show_batches_bar=False, shuffle=True, scheduler=None,
                     print_every=1, n_grad_accums=1, use_multi_gpus=False):
    if tensorboard:
        writer = SummaryWriter()
    losses = []
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=shuffle)
    if test_data is not None:
        test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)
    if use_multi_gpus:
        orig_model = model
        model = nn.DataParallel(model)
    for epoch in tqdm(range(num_epochs)):
        model = model.train()
        batch_losses = []
        batch_correct = []
        iterator = tqdm(train_loader, leave=False) if show_batches_bar else train_loader
        for i, (X, y) in enumerate(iterator):
            X = X.to(device)
            y = y.to(device)
            yhat = model(X)
            loss = loss_fn(yhat, y).mean() # mean is for multi-gpu losses, avg them together
            loss /= n_grad_accums # rescale loss for grad accumulation
            loss.backward()
            if i % n_grad_accums == 0: # only step when every n_grad_accums grad updates
                opt.step()
                opt.zero_grad()
            batch_losses.append(float(loss)* batch_size)
            batch_correct.append(float((yhat.argmax(dim=1) == y).sum().cpu()))
        train_loss = sum(batch_losses) / len(train_data)
        train_acc = sum(batch_correct) / len(train_data)
        losses.append(train_loss)
        
        if test_data is not None:
            model = model.eval()
            opt.zero_grad()
            batch_losses = []
            batch_correct = []
            iterator = tqdm(test_loader, leave=False) if show_batches_bar else test_loader
            for X, y in iterator:
                X = X.to(device)
                y = y.to(device)
                yhat = model(X)
                loss = loss_fn(yhat, y).mean()
                batch_losses.append(float(loss) * batch_size)
                batch_correct.append(float((yhat.argmax(dim=1) == y).sum().cpu()))
            test_loss = sum(batch_losses) / len(test_data)
            test_acc = sum(batch_correct) / len(test_data)
        else:
            test_loss = -999
            test_acc = -999
            
        if scheduler is not None:
            scheduler.step()

        if tensorboard:
            writer.add_scalar("Training Loss", train_loss, epoch+1)
            writer.add_scalar("Training Accuracy", train_acc, epoch+1)
            writer.add_scalar("Test Loss", test_loss, epoch+1)
            writer.add_scalar("Test Accuracy", test_acc, epoch+1)
        if print_stats and epoch % print_every == 0:
            s1 = f'epoch: {epoch: <3}   ' 
            s2 = f'train loss: {round(train_loss, 4): <6}   train acc: {round(train_acc, 4): <6}   ' 
            s3 = f'test loss: {round(test_loss, 4): <6}   test acc: {round(test_acc, 4): <6}'
            print(s1 + s2 + s3)
    if plot_loss:
        plt.plot(range(len(losses)), losses)
        plt.show()
    if tensorboard:
        writer.flush()
        writer.close()
    model = model if not use_multi_gpus else orig_model
    return model

class Dataset:
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [ ]:
# def text_from_iter(data_iter):
#     corpus = []
#     for text in data_iter:
#         corpus.append(text)
#     return corpus

# train_iter = torchtext.datasets.WikiText103(split='train')
# val_iter = torchtext.datasets.WikiText103(split='valid')
# test_iter = torchtext.datasets.WikiText103(split='test')

# train_text = text_from_iter(train_iter) + text_from_iter(val_iter)
# test_text = text_from_iter(test_iter)

# tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
# train_tokens = (tokenizer(doc) for doc in tqdm(train_text))
# test_tokens = (tokenizer(doc) for doc in tqdm(test_text))

# vocab = torchtext.vocab.build_vocab_from_iterator(train_tokens, max_tokens=10000, specials=['<unk>', '<pad>'], min_freq=50)
# vocab.set_default_index(vocab["<unk>"])
# stoi = vocab.get_stoi()
# itos = vocab.get_itos()

# max_tokens = 256
# train_tokens = [doc[:max_tokens] for doc in train_tokens if len(doc) > 5]
# test_tokens = [doc[:max_tokens] for doc in test_tokens if len(doc) > 5]

# Machine Translation: English to French

In [ ]:
nlp_en = spacy.load('en_core_web_sm')
nlp_fr = spacy.load('fr_core_news_sm')
tokenizer_en = torchtext.data.utils.get_tokenizer(nlp_en)
tokenizer_fr = torchtext.data.utils.get_tokenizer(nlp_fr)

In [ ]:
path = Path.home() / 'data' / 'machine-translation' / 'eng-fra.txt'
text = path.read_text().split('\n')
text = [t.replace('\u202f', '').split('\t') for t in text]
text = [doc for doc in text if len(doc) == 2]
len(text), text[:10]

In [ ]:
def apply_tokenizer(doc, tokenizer):
    return [token.text for token in tokenizer(doc)]

def get_tokens(doc):
    tokens_en = apply_tokenizer(doc[0], tokenizer_en)
    tokens_fr = apply_tokenizer(doc[1], tokenizer_fr)
    return tokens_en, tokens_fr

with Pool(processes=8) as pool:
    tokens = [x for x in tqdm(pool.imap(get_tokens, text), total=len(text))]

In [ ]:
max_tokens = 10

tokens_en = [toks for toks, _ in tokens if len(toks) <= max_tokens and len(_) <= max_tokens]
tokens_fr = [toks for _, toks in tokens if len(toks) <= max_tokens and len(_) <= max_tokens]
len(tokens_en), len(tokens_fr)

In [ ]:
list(zip(tokens_en, tokens_fr))[:10]

In [ ]:
vocab_en = torchtext.vocab.build_vocab_from_iterator(tokens_en, specials=['<unk>', '<pad>', '<bos>', '<eos>'])
vocab_en.set_default_index(vocab_en["<unk>"])
stoi_en = vocab_en.get_stoi()
itos_en = vocab_en.get_itos()

vocab_fr = torchtext.vocab.build_vocab_from_iterator(tokens_fr, specials=['<unk>', '<pad>', '<bos>', '<eos>'])
vocab_fr.set_default_index(vocab_fr["<unk>"])
stoi_fr = vocab_fr.get_stoi()
itos_fr = vocab_fr.get_itos()

len(vocab_en), len(vocab_fr)

In [ ]:
nums_en = [vocab_en(toks) for toks in tokens_en]
nums_fr = [vocab_fr(toks) for toks in tokens_fr]

In [ ]:
def pad_tokens(nums, seq_len, stoi, pad_token='<pad>'):
    nums = nums + [stoi[pad_token]] * (seq_len - len(nums))
    return nums

seq_len = max_tokens
nums_en_padded = [pad_tokens(num, seq_len, stoi_en) for num in nums_en]
nums_fr_padded = [pad_tokens(num, seq_len, stoi_fr) for num in nums_fr]

In [ ]:
nums_en_padded[0], nums_fr_padded[0]

# Seq2Seq Models

- Learnings a mapping from a sequence $x_1,\cdots,x_T$ to a new sequence $y_1,\cdots,y_{T'}$.
- The stereotypical application is machine translation, where text in one language is translated to language in another language. For example, translating French `le chat est noir` to English `the cat is black`. 
- Other examples include named entity recognition (NER), part of speech tagging (POS), or speech recognition (audio to text).
- Typically an encoder-decoder architecture is used. The **encoder** is a sequence model (e.g. an RNN) that takes a sequence and outputs a hidden state vector. This, along with the output sequence, then gets fed to a **decoder**, a different sequence model that uses the encoder hidden states and the output sequences to produce a predition sequence.

![Seq2Seq](https://pytorch.org/tutorials/_images/seq2seq.png)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, rnn):
        super().__init__()
        self.hidden_size = hidden_size
        self.emb = nn.Embedding(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.rnn = rnn(hidden_size, hidden_size, batch_first=True)
    
    def forward(self, x, h):
        x = self.emb(x)
        x = self.relu(x)
        x, h = self.rnn(x, h)
        return x, h
    
    def init_hidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size).to(device)
    
class Decoder(nn.Module):
    def __init__(self, output_size, hidden_size, rnn):
        super().__init__()
        self.hidden_size = hidden_size
        self.emb = nn.Embedding(output_size, hidden_size)
        self.relu = nn.ReLU()
        self.rnn = rnn(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, y, h):       
        y = self.emb(y)
        y = self.relu(y)
        y, h = self.rnn(y, h)
        y = self.fc(y)
        return y, h
    
    def init_hidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size).to(device)
    
class Seq2SeqModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, bos, rnn=nn.GRU):
        super().__init__()
        self.encoder = Encoder(input_size, hidden_size, rnn)
        self.decoder = Decoder(output_size, hidden_size, rnn)
        self.output_size = output_size
        self.bos = bos
    
    def forward(self, X, h, Y=None):
        _, h = self.encoder(X, h)
        seq_len, batch_size = X.shape[1], X.shape[0]
        Y = Y.permute(1, 0) if Y is not None else None
        Yhat = torch.empty(seq_len, batch_size, self.output_size)
        y_prev = torch.tensor([self.bos] * batch_size).reshape(batch_size, 1).to(device)
        for i in range(seq_len):
            y_out, h = self.decoder(y_prev, h)
            y_prev = y_out.argmax(dim=-1).detach()
            # y_prev = y_prev if ? else Y[i]
            Yhat[i] = y_out.permute(1, 0, 2)
        Yhat = Yhat.permute(1, 0, 2)
        return Yhat, h
    
    def init_hidden(self, batch_size):
        return self.encoder.init_hidden(batch_size)

In [ ]:
vocab_size = 1000
X = torch.randint(0, vocab_size, size=(128,10))
Y = torch.randint(0, vocab_size, size=(128,10))

model = Seq2SeqModel(vocab_size, 50, vocab_size, bos=1)
h = model.init_hidden(128)
Yhat, h = model(X, h, Y)
X.shape, h.shape, Y.shape, Yhat.shape

In [ ]:
y = Y.reshape(128 * 10,)
yhat = Yhat.reshape(128 * 10, -1)
loss_fn = nn.CrossEntropyLoss()
loss_fn(yhat, y)

In [ ]:
def train_seq2seq(train_data, model, opt, loss_fn, test_data=None, num_epochs=10, plot_loss=True, batch_size=32,
                  tensorboard=False, print_stats=True, show_batches_bar=False, shuffle=True, scheduler=None,
                  print_every=1, n_grad_accums=1, use_multi_gpus=False, grad_clip=None):
    if tensorboard:
        writer = SummaryWriter()
    losses = []
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=shuffle)
    if test_data is not None:
        test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)
    if use_multi_gpus:
        orig_model = model
        model = nn.DataParallel(model)
       
    for epoch in tqdm(range(num_epochs)):
        model = model.train()
        batch_losses = []
        iterator = tqdm(train_loader, leave=False) if show_batches_bar else train_loader
        for i, (X, Y) in enumerate(iterator):
            X = X.to(device).long()
            Y = Y.to(device).long()
            h = model.init_hidden(batch_size)
            Yhat, h = model(X, h, Y=Y)
            bs, bptt = Y.shape[0], Y.shape[1]
            y = Y.reshape(bs * bptt,)
            yhat = Yhat.reshape(bs * bptt, -1)
            loss = loss_fn(yhat, y).mean() # mean is for multi-gpu losses, avg them together
            loss /= n_grad_accums # rescale loss for grad accumulation
            loss.backward()
            if grad_clip is not None:
                nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
            if i % n_grad_accums == 0: # only step when every n_grad_accums grad updates
                opt.step()
                opt.zero_grad()
            batch_losses.append(float(loss)* batch_size)
        train_loss = sum(batch_losses) / len(train_data)
        losses.append(train_loss)
        
        if test_data is not None:
            model = model.eval()
            opt.zero_grad()
            batch_losses = []
            iterator = tqdm(test_loader, leave=False) if show_batches_bar else test_loader
            for X, Y in iterator:
                X = X.to(device)
                Y = Y.to(device)
                h = model.init_hidden(batch_size)
                Yhat, h = model(X, h, Y=Y)
                bs, bptt = Y.shape[0], Y.shape[1]
                y = Y.reshape(bs * bptt,)
                yhat = Yhat.reshape(bs * bptt, -1)
                loss = loss_fn(yhat, y).mean()
                batch_losses.append(float(loss) * batch_size)
            test_loss = sum(batch_losses) / len(test_data)
        else:
            test_loss = -999
            
        if scheduler is not None:
            scheduler.step()

        if tensorboard:
            writer.add_scalar("Training Loss", train_loss, epoch+1)
            writer.add_scalar("Test Loss", test_loss, epoch+1)
        if print_stats and epoch % print_every == 0:
            s1 = f'epoch: {epoch: <3}   ' 
            s2 = f'train loss: {round(train_loss, 4): <6}   test loss: {round(test_loss, 4): <6}' 
            print(s1 + s2)
    if plot_loss:
        plt.plot(range(len(losses)), losses)
        plt.show()
    if tensorboard:
        writer.flush()
        writer.close()
    model = model if not use_multi_gpus else orig_model
    return model

In [ ]:
idx_train, idx_test = train_test_split(range(len(nums_en_padded)), train_size=0.9, shuffle=True, random_state=seed)

X = torch.tensor(nums_en_padded).long()
Y = torch.tensor(nums_fr_padded).long()

X_train = X[idx_train]
Y_train = Y[idx_train]
X_test = X[idx_test]
Y_test = Y[idx_test]

X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

In [ ]:
train_data = Dataset(X_train, Y_train)
test_data = Dataset(X_test, Y_test)

In [ ]:
vocab_size_en = len(vocab_en)
vocab_size_fr = len(vocab_fr)
emb_size = 100
bos = stoi_fr['<bos>'] # output <bos> token

In [ ]:
model = Seq2SeqModel(vocab_size_en, emb_size, vocab_size_fr, bos).to(device)
loss_fn = nn.CrossEntropyLoss()
opt = get_optimizer(model, optimizer='adam', lr=0.001, weight_decay=0.)
model = train_seq2seq(train_data[:100*128], model, opt, loss_fn, num_epochs=10, batch_size=128, grad_clip=None, 
                      test_data=test_data[:100*128], print_every=1)

In [ ]:
def predict(tokens, vocab_en, vocab_fr):
    pass

**Image to Text Models**
- Maps an image to a sequence of text output. The stereotypical example is image captioning, where one feeds in an image and asks the model to "describe" what the image is in words.
- Typically a CNN model is used to get a feature map of the image, which is then fed as an input into a sequence model decoder to output text.

**Beam Search**
- In a seq2seq model, the model tries to learn $p(y_1,\cdots,y_{T'}|x_1,\cdots,x_T)$. In many applications, one doesn't want to sample from this distribution, but to find the *most likely* sequence $\hat y_1, \cdots, \hat y_{T'}$, e.g. in machine translation where one wants the *best* possible translation.
- The naive way of outputing a prediction is a "greedy search". This approach just uses the argmax of each predicted $\hat y_t$ to make a prediction. Greedy searches aren't generally optimal, in the sense of getting the argmax of the *joint distribution*. In the case of seq2seq, the greedy search can overly focus on the earlier inputs, causing it to miss the big picture.
- Beam search is a way around this. It's an approximate algorithm for finding the max of a set of sequences sampled from a joint distribution, i.e. finding $\hat y = \text{argmax} p(y_1,\cdots,y_{T'}|x_1,\cdots,x_T)$.

**Attention Models**
- Vanilla seq2seq models tend not to work as well for longer sequences, e.g. 30+ token sequences. This is in essence because they have to remember the entire input to pass into the decoder and decide what the output should be. 
- Attention models get around this problem by allowing the output to focus just on the local input context (e.g. the first few words) by using a learned "attention layer" to learn the probabilities of any one input affecting that output. That way, the decoder can only focus on a specific range of inputs, not the whole sequence.
- Example: Consider the translation `Jane visite l'Afrique en septembre` -> `Jane visits Africa in September`. In predicting the output token `Jane`, the model will probably want to look most at the beginning of the input, i.e. `Jane`. So maybe the input tokens should be weighted, e.g. `Jane: 0.9, visite: 0.6, l'Afrique: 0.2, en: 0.1, septembre: 0.1`. That is, the model should focus 90% of its weight on the first word `Jane` when predicting the output word `Jane`.
- Blah

**Teacher Forcing**